# Python 101
## XIII. Databases and connections

---

## I. SQLite3

DB API for lightweight disk-based database. It is a serverless db engine to store structured data in separate files on the disk or in memory.

In [ ]:
import sqlite3

### 1. Basic workflow 
- Create a connection object to represent the db (it also creates the target file if is not existing)

In [ ]:
conn = sqlite3.connect('data/example.db')

- create a cursor to an existing db connection to execute SQL queries

In [ ]:
c = conn.cursor()

- execute SQL queries

In [ ]:
c.execute('''CREATE TABLE IF NOT EXISTS test
             (name text,
              email text,
              balance integer);''')
c.execute('''INSERT INTO test VALUES ('John', 'john@doe.com', 100);''')

- commit changes

In [ ]:
conn.commit()

- get query results

In [ ]:
c.execute('''SELECT * FROM test;''')
c.fetchone()

- close connection

In [ ]:
conn.close()

### 2. Using Python variables

Instead of writing SQL queries manually, automated query generation is supported. Instead of string formatting which is susceptible for SQL injection attacks we can use sqlite3's own parameter substitution solution.

In [ ]:
import sqlite3
conn = sqlite3.connect('data/example.db')
c = conn.cursor()

- insecure method

In [ ]:
username = 'John'
c.execute(f"SELECT * FROM test WHERE name = '{username}';")
c.fetchone()

- safe method

In [ ]:
username = ('John',)
c.execute('SELECT * FROM test WHERE name=?', username)
c.fetchone()

In [ ]:
c.execute('SELECT * FROM test WHERE name=:user', {'user': 'John'})
c.fetchone()

- multiple values can be used in the same time

In [ ]:
values = [
    ('Joe', 'joe@doe.com', 200),
    ('Bill', 'bill@wildstallins.com', 5),
    ('Ted', 'ted@wildstallins.com', 2),
    ('Death', 'death@wildstallins.com', 1000)
]
r = c.executemany('INSERT INTO test VALUES (?, ?, ?)', values)
print(f'Inserted {r.rowcount} lines')
conn.commit()

### 3. Fetching options

In [ ]:
query = 'SELECT * FROM test;'

- first item

In [ ]:
c.execute(query)
c.fetchone()

- gather all result at the same time

In [ ]:
c.execute(query)
c.fetchall()

- iterate on results

In [ ]:
for row in c.execute(query):
    print(row)

In [ ]:
conn.close()

### 4. Error handling

DB operations should be inside try-except blocks and changes made should be rolled back in case of errors. Sqlite3 has it's own exception subclass.

In [ ]:
try:
    conn = sqlite3.connect(':memory:')
    c = conn.cursor()
    c.execute('''CREATE TABLE IF NOT EXISTS test
                 (name text,
                  email text,
                  balance integer);''')
    c.execute('''INSERT INTO test VALUES ('John', 'john@doe.com', 100);''')
    raise sqlite3.Error('Test error')
    conn.commit()
except sqlite3.Error as e:
    print(f'Exception occured with the following error: {e}')
    conn.rollback()
finally:
    print('Test query result:', c.execute('SELECT * FROM test;').fetchall())
    conn.close()

### 5. Exercise: Create and fill a table from `data/matching.csv`

matching table columns:
- id - integer (primary key)
- group_id - integer
- name - text
- val1 - integer
- val2 - integer

---

## II. SQLAlchemy

SQLAlchemy is a Python SQL toolkit which creates a unified interface to many different db engines. It supports sqlite3, mysql, postresql, etc.

In [ ]:
from sqlalchemy import create_engine

### 1. Basic workflow

- create an engine

In [ ]:
engine = create_engine('sqlite:///data/example.db')

- connect

In [ ]:
connection = engine.connect()

- query

In [ ]:
result = connection.execute("select * from test;")
for row in result:
    print(dict(row))

- close connection

In [ ]:
connection.close()

### 2. SQL Expression Language

SqlAlchemy provides a direct mapping of the db objects to python objects. These objects will be used to interact with the db.

In [ ]:
from sqlalchemy import Table, Column, Integer, String, MetaData

- create engine

In [ ]:
engine = create_engine('sqlite:///data/example.db')

- generate metadata for the tables

In [ ]:
metadata = MetaData()

- register users table

In [ ]:
users = Table('users', metadata,
    Column('id', Integer, primary_key=True),
    Column('name', String),
    Column('fullname', String),
)

- create tables

In [ ]:
metadata.create_all(engine)

- preparing insertion

In [ ]:
ins = users.insert()
str(ins)

- setting the params

In [ ]:
ins = users.insert().values(name='jack', fullname='Jack Jones')
str(ins)

- executing query

In [ ]:
conn = engine.connect()

In [ ]:
result = conn.execute(ins)
result.rowcount

- multiple insertion is available as well

In [ ]:
multiresult = conn.execute(users.insert(), [
    {'name': 'jill', 'fullname': 'jill doe'},
    {'name': 'james', 'fullname': 'james doe'},
    {'name': 'john', 'fullname': 'john doe'},
    {'name': 'joe', 'fullname': 'joe doe'},
])

multiresult.rowcount

- checking results

In [ ]:
for row in conn.execute(users.select()):
    print(dict(row))


In [ ]:
for row in conn.execute(users.select().where(users.columns['id'] > 2)):
    print(dict(row))

In [ ]:
conn.close()

---

## III. Pandas

Pandas is a data structure and data analysis tool which has built-in support for data gathering from different datasources. Reading and writing data from databases using sql queries is supported through SQLAlchemy.

### 1. Reading data from db with `pandas.read_sql`
`pandas.read_sql` provides a comfortable interface for reading data from a db using an SQLAlchemy engine and an sql query.

In [ ]:
import pandas as pd
from sqlalchemy import create_engine

In [ ]:
engine = create_engine('sqlite:///data/example.db', echo=False)

- read entire table

In [ ]:
pd.read_sql('test', con=engine)

- read sql query results into table

In [ ]:
pd.read_sql('SELECT email, balance FROM test WHERE balance < 100;', con=engine)

In [ ]:
pd.read_sql('SELECT email, balance FROM test WHERE email=?;', con=engine, params=('joe@doe.com',))

In [ ]:
pd.read_sql('SELECT email, balance FROM test WHERE email=:mail;', con=engine, params={'mail': 'joe@doe.com'})

### 2. Writing data to db with `pandas.to_sql`
`pandas.to_sql` exports data into a db table

- Create a connection

In [ ]:
engine = create_engine('sqlite:///data/example.db', echo=False)

- get data

In [ ]:
df = pd.read_csv('data/cars.csv')
df.head()

- save into sql table

In [ ]:
df.to_sql('cars', con=engine, if_exists='replace')
# check
pd.read_sql('cars', con=engine).tail()

- append to existing table

In [ ]:
df_extra = pd.DataFrame([{'brand': 'ram', 'group': 'fiat-chrysler'}], index=[36])
df_extra

In [ ]:
df_extra.to_sql('cars', con=engine, if_exists='append')
pd.read_sql('cars', con=engine).tail()

---

## III. Exercises

### 1. Select the brands from the vw group!

`CARS(index INTEGER, brand TEXT, group TEXT)`

### 2. Create user login handling!
#### A. Create a new table in the `example.db` called passwords
- it has two columns: 
    - id (int) foreign key (users.id)
    - password (int)

#### B. Write a function which registers a new user
- `register(username, full_name, password)`
- insert user's name, fullname to users table
- insert password's hash to passwords table

Hint:
- user the `hash()` function to generate hash value

#### C. Write a function for logging in a user in
- `login(username, password)`
- return `True` if credentials were correct